#### ???
* Что такое синтаксическое дерево
* Почему регулярные выражения так называются
* Чем компилятор отличается от интерперетора

In [1]:
from datetime import datetime
start_time = datetime.now()
def time_elapsed():
    global start_time
    print(datetime.now() - start_time)

#### Что мы будем использовать?

Python и его библиотеки
* https://docs.python.org/3/library/dis.html
* https://docs.python.org/3/library/ast.html
* https://greentreesnakes.readthedocs.io/en/latest


* [lark-parser](https://github.com/lark-parser/lark) - библиотека для построения парсера по спецецификации
* [astpretty](https://github.com/asottile/astpretty) - вывод ast на экран


* Утилиты:
* [funcy](https://github.com/Suor/funcy)
* [more-itertools](https://github.com/more-itertools/more-itertools)

*Отступление*
* [facesofopensource.com](https://www.facesofopensource.com/guido-van-rossum-2/)


#### Нужно ли это разработчикам?
???

domain specific language

### Определения

[Язык программирования](https://ru.wikipedia.org/wiki/%D0%AF%D0%B7%D1%8B%D0%BA_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F) -  формальный язык, предназначенный для записи компьютерных программ с набором набор лексических, синтаксических и семантических правил.

[Формальный язык](https://ru.wikipedia.org/wiki/%D0%A4%D0%BE%D1%80%D0%BC%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D1%8F%D0%B7%D1%8B%D0%BA) - множество слов (цепочек символов) над конечным алфавитом, определенных посредсвом некторых правил


{0..9}

12389483

1347324987

1343

265756


### [Иерархия классов](https://en.wikipedia.org/wiki/Chomsky_hierarchy)
![Chomsky-hierarchy](assets/640px-Chomsky-hierarchy.svg.png)

### Процесс компиляции

![flow](assets/parser_flow.diag.svg)



### Токенизация

In [2]:
from io import BytesIO
import tokenize

In [3]:
tokenize.tokenize?

In [7]:
'1323   ksjjdfkjsjdh hkdsfjhs'.split()

['1323', 'ksjjdfkjsjdh', 'hkdsfjhs']

In [6]:
data = BytesIO(b"""
if x > y:
  r = x   -   y * 2
  
else:
  r = xxx + y
""").readline

for tok in tokenize.tokenize(data):
    print(tok)

TokenInfo(type=62 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line='')
TokenInfo(type=61 (NL), string='\n', start=(1, 0), end=(1, 1), line='\n')
TokenInfo(type=1 (NAME), string='if', start=(2, 0), end=(2, 2), line='if x > y:\n')
TokenInfo(type=1 (NAME), string='x', start=(2, 3), end=(2, 4), line='if x > y:\n')
TokenInfo(type=54 (OP), string='>', start=(2, 5), end=(2, 6), line='if x > y:\n')
TokenInfo(type=1 (NAME), string='y', start=(2, 7), end=(2, 8), line='if x > y:\n')
TokenInfo(type=54 (OP), string=':', start=(2, 8), end=(2, 9), line='if x > y:\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(2, 9), end=(2, 10), line='if x > y:\n')
TokenInfo(type=5 (INDENT), string='  ', start=(3, 0), end=(3, 2), line='  r = x   -   y * 2\n')
TokenInfo(type=1 (NAME), string='r', start=(3, 2), end=(3, 3), line='  r = x   -   y * 2\n')
TokenInfo(type=54 (OP), string='=', start=(3, 4), end=(3, 5), line='  r = x   -   y * 2\n')
TokenInfo(type=1 (NAME), string='x', start=(3, 6), end=(3, 7),

#### Что такое AST?

`1 + 2 * 3`

```
     +
    / \
   1   *
      / \
     2   3
```

```
EXPR -> NUM | EXPR + EXPR | EXPR * EXPR
NUM -> 0|1|2|...|9
```


#### AST vs Parse Tree
*опционально*

In [8]:
import ast
import astpretty

* https://docs.python.org/3/library/ast.html
* https://greentreesnakes.readthedocs.io/en/latest

In [28]:
ast.parse?

In [18]:
node = ast.parse("""
if x > y:
  r = x - y * 2
  x =1
else:
  r = x + y
x = 1
""")
print(node)

In [19]:
# TODO: work with `node`

In [25]:
node.body[0].body[0].value.op

In [26]:
astpretty.pprint(node)

Module(
    body=[
        If(
            lineno=2,
            col_offset=0,
            end_lineno=6,
            end_col_offset=11,
            test=Compare(
                lineno=2,
                col_offset=3,
                end_lineno=2,
                end_col_offset=8,
                left=Name(lineno=2, col_offset=3, end_lineno=2, end_col_offset=4, id='x', ctx=Load()),
                ops=[Gt()],
                comparators=[Name(lineno=2, col_offset=7, end_lineno=2, end_col_offset=8, id='y', ctx=Load())],
            ),
            body=[
                Assign(
                    lineno=3,
                    col_offset=2,
                    end_lineno=3,
                    end_col_offset=15,
                    targets=[Name(lineno=3, col_offset=2, end_lineno=3, end_col_offset=3, id='r', ctx=Store())],
                    value=BinOp(
                        lineno=3,
                        col_offset=6,
                        end_lineno=3,
                        

In [27]:
time_elapsed()

0:22:54.515771


#### Пробуем написать свой парсер

In [29]:
import more_itertools
from pprint import pprint

In [30]:
# LL(*) -- top-down
# LR -- bottom-up 

In [31]:
print(tokens := '1 + 2 + 3 + 4'.split())
stream = more_itertools.peekable(tokens)

['1', '+', '2', '+', '3', '+', '4']


In [33]:
print('peek', stream.peek())
print('next', next(stream))
print('peek', stream.peek())
print('peek', stream.peek())
print('next', next(stream))
print('next', next(stream))

peek +
next +
peek 3
peek 3
next 3
next +


In [34]:
def mynode(typ, val):
    assert val is not None
    return {
        'typ': typ,
        'val': val,
    }

####  Определим грамматику
```
EXPR ->  NUM [+-] EXPR | NUM
NUM -> 0|1|2|3|..|0
```

In [35]:
def consume_int(stream):
    if stream.peek('').isdigit():
        return int(next(stream))
    return None

def consume_tok(tokens, stream):
    tok = stream.peek(None)
    if tok is not None and tok in tokens:
        return next(stream)
    return None

def parse_expr(stream):
    lhs = mynode('num', consume_int(stream))

    op = consume_tok(['+', '-'], stream)
    if op is None:
        return lhs
    rhs = parse_expr(stream)
    assert rhs is not None
    return mynode('op', [lhs, op, rhs])

In [36]:
node = parse_expr(more_itertools.peekable('1 + 2 + 3 + 4'.split()))
pprint(node)

{'typ': 'op',
 'val': [{'typ': 'num', 'val': 1},
         '+',
         {'typ': 'op',
          'val': [{'typ': 'num', 'val': 2},
                  '+',
                  {'typ': 'op',
                   'val': [{'typ': 'num', 'val': 3},
                           '+',
                           {'typ': 'num', 'val': 4}]}]}]}


In [37]:
def evaluate(node):
    ops = {
        '+': lambda x, y: x + y,
        '-': lambda x, y: x - y,
    }
    if node['typ'] == 'op':
        lchild, op, rchild =  node['val']
        return ops[op](evaluate(lchild), evaluate(rchild))
    if node['typ'] == 'num':
        return node['val']
    assert 'unknown type', node


In [38]:
evaluate(node)

10

In [39]:
node = parse_expr(more_itertools.peekable('1 + 2 - 3 + 4'.split()))
pprint(node)

{'typ': 'op',
 'val': [{'typ': 'num', 'val': 1},
         '+',
         {'typ': 'op',
          'val': [{'typ': 'num', 'val': 2},
                  '-',
                  {'typ': 'op',
                   'val': [{'typ': 'num', 'val': 3},
                           '+',
                           {'typ': 'num', 'val': 4}]}]}]}


In [ ]:
1 + (2 - (3 + 4))

#### Что выведет `evaluate` ?
```
  +
 / \
1   -
   / \
  2   +
     / \
    3   4
```

In [41]:
evaluate(node)

-4

#### Грамматика версия 2
```
EXPR -> NUM | EXPR OP NUM
NUM -> 0..9
```

In [42]:
def parse_expr2(stream):
    lhs = parse_expr2(stream)
    
    op = consume_tok(['+', '-'], stream)
    if op is None:
        return lhs
    rhs = mynode('num', consume_int(stream))
    assert rhs is not None
    return mynode('op', [lhs, op, rhs])

#### DO NOT RUN !!!

In [43]:
node = parse_expr2(more_itertools.peekable('1 + 2 - 3 + 4'.split()))
pprint(node)

RecursionError: maximum recursion depth exceeded

#### Грамматика версия 3 
```
EXPR -> NUM {[+-] NUM}+
NUM -> 0..9
```

In [44]:
def parse_expr3(stream):
    lhs = mynode('num', consume_int(stream))
        
    assert lhs is not None
    res = lhs
    
    while True:
        op = consume_tok(['+', '-'], stream)
        if op is None:
            break

        rhs = mynode('num', consume_int(stream))
        assert rhs is not None
        
        res = mynode('op', [res, op, rhs])
    return res


In [45]:
node = parse_expr3(more_itertools.peekable('1 + 2 - 3 + 4'.split()))
pprint(node)

{'typ': 'op',
 'val': [{'typ': 'op',
          'val': [{'typ': 'op',
                   'val': [{'typ': 'num', 'val': 1},
                           '+',
                           {'typ': 'num', 'val': 2}]},
                  '-',
                  {'typ': 'num', 'val': 3}]},
         '+',
         {'typ': 'num', 'val': 4}]}


#### Результат
```
        + 
       / \
      -   4
     / \
    +   3
   / \ 
  1   2 
 
```

In [46]:
evaluate(node)

4

In [47]:
time_elapsed()

0:42:55.563700


### Что делать с AST?

* https://docs.python.org/3/library/dis.html
* [Understanding Python Bytecode](https://towardsdatascience.com/understanding-python-bytecode-e7edaae8734d)
* [How to patch Python bytecode](https://rushter.com/blog/python-bytecode-patch/)

In [48]:
import dis

In [49]:
compile?

In [50]:
node = ast.parse("""
x = a + b
if x > 0:
    print(x)
""")

In [51]:
code = compile(node, '<foo>', "exec")

In [52]:
code

<code object <module> at 0x7fb41ab4d660, file "<foo>", line 2>

In [53]:
def pcodeobj(code):
    for attr in dir(code):
        if attr.startswith('co_'):
            print("\t%s = %s" % (attr, code.__getattribute__(attr)))


In [55]:
pcodeobj(code)

	co_argcount = 0
	co_cellvars = ()
	co_code = b'e\x00e\x01\x17\x00Z\x02e\x02d\x00k\x04r\x18e\x03e\x02\x83\x01\x01\x00d\x01S\x00'
	co_consts = (0, None)
	co_filename = <foo>
	co_firstlineno = 2
	co_flags = 64
	co_freevars = ()
	co_kwonlyargcount = 0
	co_lnotab = b'\x08\x01\x08\x01'
	co_name = <module>
	co_names = ('a', 'b', 'x', 'print')
	co_nlocals = 0
	co_posonlyargcount = 0
	co_stacksize = 2
	co_varnames = ()


In [56]:
dis.dis(code)

  2           0 LOAD_NAME                0 (a)
              2 LOAD_NAME                1 (b)
              4 BINARY_ADD
              6 STORE_NAME               2 (x)

  3           8 LOAD_NAME                2 (x)
             10 LOAD_CONST               0 (0)
             12 COMPARE_OP               4 (>)
             14 POP_JUMP_IF_FALSE       24

  4          16 LOAD_NAME                3 (print)
             18 LOAD_NAME                2 (x)
             20 CALL_FUNCTION            1
             22 POP_TOP
        >>   24 LOAD_CONST               1 (None)
             26 RETURN_VALUE


### Почему байткод быстрее AST-интепретации?

* простой, меньше накладных расходов в интепретаторе
* легче оптимизировать
* дружественный к кешу

### Ruby 1.8 vs 1.9

http://rubychan.de/share/yarv_speedups.html


*Отступление*

Principle of locality


### Процесс компиляции (еще раз)

![flow](assets/parser_flow.diag.svg)



### Rust Example
https://blog.rust-lang.org/2016/04/19/MIR.html

In [57]:
time_elapsed()

0:55:17.501666


### Let's Code ...

In [58]:
import typing
import dis
import ast

from types import CodeType

import struct

from io import BytesIO
from dataclasses import dataclass
from collections import defaultdict

from lark import Lark, Transformer, v_args
from funcy import collecting

https://github.com/lark-parser/lark

#### Что напишем?
* https://en.wikipedia.org/wiki/Euclidean_algorithm
* https://en.wikipedia.org/wiki/Fibonacci_number

* Наш язык поддержит целые числа и операции `+ - * /`
* Операторы сравнения `<, >, >=, ...`
* Условные и безусловные переходы (aka `if goto`)
* Ввод-вывод

In [59]:
def gdc(a, b):
    while a != b:
        if a > b:
            a -= b
        else:
            b -= a
    return a

assert gdc(12, 8) == 4
assert gdc(56, 42) == 14
assert gdc(1, 10) == 1
del gdc

In [60]:
def fib(n):
    a = 1
    b = 1
    for _ in range(n):
        c = a + b
        a = b
        b = c
    return a 

assert fib(1) == 1
assert fib(2) == 2
assert fib(3) == 3
assert fib(4) == 5
assert fib(5) == 8
assert fib(6) == 13
del fib

In [ ]:

./foo.mlg

In [61]:
text = """
#!/usr/bin/env my_lang

ask n 
set a to 1
set b to 1
set i to 1
label head
    set i to i + 1
    set c to a + b
    set a to b
    set b to c 
    goto head if i < n
    
show a

"""



In [63]:
time_elapsed()

1:11:00.930109


In [68]:
grammar = """
    start: _NL* statement (_NL+ statement )* _NL*
    
    ?statement: "ask" NAME -> ask 
                | "set" NAME "to" expr -> assign
                | "label" NAME -> label
                | "show" NAME -> show
                | "goto" NAME "if" expr -> goto

    ?expr: sum
           | sum CMP sum -> binop  
    
    ?sum: product
        | sum ADDOP product -> binop

    CMP: "="|">="|"<="|"/="|">"|"<"
    ADDOP: "+"|"-"
    MULOP: "*"|"/"
    
    ?product: atom
        | product MULOP atom -> binop
        
    ?atom: NUMBER           -> number
         | NAME             -> var
         | "(" sum ")"

    %import common.CNAME -> NAME
    %import common.NEWLINE -> _NL
    %import common.NUMBER
    %import common.WS_INLINE
    %ignore WS_INLINE
"""

In [69]:
parser = Lark(grammar, parser='lalr')

In [71]:
parser.parse(text)

Tree(start, [Tree(ask, [Token(NAME, 'n')]), Tree(assign, [Token(NAME, 'a'), Tree(number, [Token(NUMBER, '1')])]), Tree(assign, [Token(NAME, 'b'), Tree(number, [Token(NUMBER, '1')])]), Tree(assign, [Token(NAME, 'i'), Tree(number, [Token(NUMBER, '1')])]), Tree(label, [Token(NAME, 'head')]), Tree(assign, [Token(NAME, 'i'), Tree(binop, [Tree(var, [Token(NAME, 'i')]), Token(ADDOP, '+'), Tree(number, [Token(NUMBER, '1')])])]), Tree(assign, [Token(NAME, 'c'), Tree(binop, [Tree(var, [Token(NAME, 'a')]), Token(ADDOP, '+'), Tree(var, [Token(NAME, 'b')])])]), Tree(assign, [Token(NAME, 'a'), Tree(var, [Token(NAME, 'b')])]), Tree(assign, [Token(NAME, 'b'), Tree(var, [Token(NAME, 'c')])]), Tree(goto, [Token(NAME, 'head'), Tree(binop, [Tree(var, [Token(NAME, 'i')]), Token(CMP, '<'), Tree(var, [Token(NAME, 'n')])])]), Tree(show, [Token(NAME, 'a')])])

In [72]:
print(parser.parse(text).pretty())

start
  ask	n
  assign
    a
    number	1
  assign
    b
    number	1
  assign
    i
    number	1
  label	head
  assign
    i
    binop
      var	i
      +
      number	1
  assign
    c
    binop
      var	a
      +
      var	b
  assign
    a
    var	b
  assign
    b
    var	c
  goto
    head
    binop
      var	i
      <
      var	n
  show	a



#### Опрделяем таргет

In [77]:
@dataclass
class PythonInst:
    mnemonic: str
    argument: typing.Any = 0
    argtype: str = None
    
    def size(self):
        return 2

    def to_binary(self, ctx, writer):
        operand = 0
        if self.argtype is None:
            operand = self.argument
        else:
            operand = ctx[self.argtype][self.argument]
        opcode = dis.opmap[self.mnemonic]
        writer.write(struct.pack('BB', opcode, operand))
        
@dataclass
class LabelInst:
    label: str

    def size(self):
        return 0

    def to_binary(self, ctx, writer):
        return None

#### stdlib

In [84]:
def read_ints():
    return tuple(int(v) for v in input('>> ').split())

def read_int():
    return int(input('> '))

In [92]:
dis.dis(compile(ast.parse("""
if x:
    foo(x, 1)
    x =1
x = 1
"""), '<foo>', 'exec'))

  2           0 LOAD_NAME                0 (x)
              2 POP_JUMP_IF_FALSE       18

  3           4 LOAD_NAME                1 (foo)
              6 LOAD_NAME                0 (x)
              8 LOAD_CONST               0 (1)
             10 CALL_FUNCTION            2
             12 POP_TOP

  4          14 LOAD_CONST               0 (1)
             16 STORE_NAME               0 (x)

  5     >>   18 LOAD_CONST               0 (1)
             20 STORE_NAME               0 (x)
             22 LOAD_CONST               1 (None)
             24 RETURN_VALUE


https://docs.python.org/3/library/dis.html
* `CALL_FUNCTION`
* `JUMP_ABSOLUTE`
* `LOAD_CONST`
* `LOAD_NAME`
* `POP_JUMP_IF_TRUE`
* `POP_TOP`
* `RETURN_VALUE`
* `STORE_NAME`
* `UNPACK_SEQUENCE`
* `BINARY_MULTIPLY`
* `BINARY_TRUE_DIVIDE`
* `BINARY_ADD`
* `BINARY_SUBTRACT`
* `COMPARE_OP`

#### Пишем визитор

In [75]:
dis.cmp_op

('<',
 '<=',
 '==',
 '!=',
 '>',
 '>=',
 'in',
 'not in',
 'is',
 'is not',
 'exception match',
 'BAD')

In [97]:
op_mapping = {
    '*': PythonInst('BINARY_MULTIPLY', 0),
    '+': PythonInst('BINARY_ADD', 0),
    '-': PythonInst('BINARY_SUBTRACT', 0),
    '<': PythonInst('COMPARE_OP', dis.cmp_op.index('<')), 
    '>': PythonInst('COMPARE_OP', dis.cmp_op.index('>')), 
}


@v_args(inline=True)
class VisitTree(Transformer):
    """
    start: _NL* statement (_NL+ statement )* _NL*
    
    ?statement: "ask" NAME -> ask 
                | "set" NAME "to" expr -> assign
                | "label" NAME -> label
                | "show" NAME -> show
                | "goto" NAME "if" expr -> goto

    ?expr: sum
           | sum CMP sum -> binop  
    
    ?sum: product
        | sum ADDOP product -> binop

    CMP: "="|">="|"<="|"/="|">"|"<"
    ADDOP: "+"|"-"
    MULOP: "*"|"/"
    
    ?product: atom
        | product MULOP atom -> binop
        
    ?atom: NUMBER           -> number
         | NAME             -> var
         | "(" sum ")"

    """
    @collecting
    def number(self, num):
        yield PythonInst('LOAD_CONST', int(num), 'const')
        
    @collecting
    def var(self, name):
        yield PythonInst('LOAD_NAME', name, 'name')
    
    @collecting
    def binop(self, lhs, op, rhs):
        yield from lhs
        yield from rhs
        yield op_mapping[op]
        
    @collecting
    def ask(self, name):
        yield PythonInst('LOAD_NAME', 'read_int', 'name')
        yield PythonInst('CALL_FUNCTION', 0)
        yield PythonInst('STORE_NAME', name, 'name')
        
    @collecting
    def show(self, *names):
        yield PythonInst('LOAD_NAME', 'print', 'name')
        for name in names:
            yield PythonInst('LOAD_NAME', name, 'name')
        yield PythonInst('CALL_FUNCTION', len(name))
        yield PythonInst('POP_TOP')

    @collecting
    def label(self, name):
        yield LabelInst(name)
    
    @collecting
    def goto(self, label, expr=None):
        yield from expr
        yield PythonInst('POP_JUMP_IF_TRUE', label, 'label')
     
    @collecting       
    def assign(self, name, expr):
        yield from expr
        yield PythonInst('STORE_NAME', name, 'name')

    @collecting
    def start(self, *statements):
        for st in statements:
            yield from st
        yield PythonInst('LOAD_CONST', 0, 'const')
        yield PythonInst('RETURN_VALUE')
    
parser = Lark(grammar, parser='lalr', transformer=VisitTree())

In [98]:
parser.parse(text)

[PythonInst(mnemonic='LOAD_NAME', argument='read_int', argtype='name'),
 PythonInst(mnemonic='CALL_FUNCTION', argument=0, argtype=None),
 PythonInst(mnemonic='STORE_NAME', argument=Token(NAME, 'n'), argtype='name'),
 PythonInst(mnemonic='LOAD_CONST', argument=1, argtype='const'),
 PythonInst(mnemonic='STORE_NAME', argument=Token(NAME, 'a'), argtype='name'),
 PythonInst(mnemonic='LOAD_CONST', argument=1, argtype='const'),
 PythonInst(mnemonic='STORE_NAME', argument=Token(NAME, 'b'), argtype='name'),
 PythonInst(mnemonic='LOAD_CONST', argument=1, argtype='const'),
 PythonInst(mnemonic='STORE_NAME', argument=Token(NAME, 'i'), argtype='name'),
 LabelInst(label=Token(NAME, 'head')),
 PythonInst(mnemonic='LOAD_NAME', argument=Token(NAME, 'i'), argtype='name'),
 PythonInst(mnemonic='LOAD_CONST', argument=1, argtype='const'),
 PythonInst(mnemonic='BINARY_ADD', argument=0, argtype=None),
 PythonInst(mnemonic='STORE_NAME', argument=Token(NAME, 'i'), argtype='name'),
 PythonInst(mnemonic='LOAD_NA

In [99]:
def create_codeobj(instructions):
    
    constants = tuple(set(
        inst.argument
        for inst in instructions
        if isinstance(inst, PythonInst) and inst.argtype == 'const'
    ))
    
    names = tuple(set(
        inst.argument
        for inst in instructions
        if isinstance(inst, PythonInst) and inst.argtype == 'name'
    ))
    context = {
        'const': {
            k: i
            for i, k in enumerate(constants)
        },
        'name': {
            k: i
            for i, k in enumerate(names)
        },
        'label': {},
    }
    
    off = 0
    for inst in instructions:
        if isinstance(inst, LabelInst):
            context['label'][inst.label]  = off
        off += inst.size()
    
    
    for inst in instructions:
        if isinstance(inst, PythonInst):
            if inst.argtype == 'label':
                assert inst.argument in context['label']
    
    w = BytesIO()

    for inst in instructions:
        inst.to_binary(context, w)
    
    return CodeType(
        0, # argcount
        0, # posonlyargcount
        0, # kwonlyargcount
        0, # nlocals
        64, # stacksize
        64, # flags
        w.getvalue(), # codestring
        constants, # constants
        names, # names
        (), # varnames
        '<ok>', # filename
        '<module>', # name
        1, # firstlineno
        b'',# lnotab
    )

In [100]:
code = create_codeobj(parser.parse(text))

In [101]:
dis.dis(code)

  1           0 LOAD_NAME                1 (read_int)
              2 CALL_FUNCTION            0
              4 STORE_NAME               4 (n)
              6 LOAD_CONST               1 (1)
              8 STORE_NAME               0 (a)
             10 LOAD_CONST               1 (1)
             12 STORE_NAME               6 (b)
             14 LOAD_CONST               1 (1)
             16 STORE_NAME               5 (i)
        >>   18 LOAD_NAME                5 (i)
             20 LOAD_CONST               1 (1)
             22 BINARY_ADD
             24 STORE_NAME               5 (i)
             26 LOAD_NAME                0 (a)
             28 LOAD_NAME                6 (b)
             30 BINARY_ADD
             32 STORE_NAME               3 (c)
             34 LOAD_NAME                6 (b)
             36 STORE_NAME               0 (a)
             38 LOAD_NAME                3 (c)
             40 STORE_NAME               6 (b)
             42 LOAD_NAME                5 (i)
   

In [107]:
time_elapsed()

1:59:18.432822


In [105]:
exec(code)

> 7
13


### Книги


Compilers:
![CompilersBook2ed](assets/CompilersBook2ed.png)

* [dragonbook](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BC%D0%BF%D0%B8%D0%BB%D1%8F%D1%82%D0%BE%D1%80%D1%8B:_%D0%BF%D1%80%D0%B8%D0%BD%D1%86%D0%B8%D0%BF%D1%8B,_%D1%82%D0%B5%D1%85%D0%BD%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D0%B8_%D0%B8_%D0%B8%D0%BD%D1%81%D1%82%D1%80%D1%83%D0%BC%D0%B5%D0%BD%D1%82%D1%8B) (aka "Компиляторы: принципы, технологии и инструменты")

* [craftinginterpreters](https://craftinginterpreters.com/contents.html)

* [Engineering: A Compiler](https://www.amazon.com/Engineering-Compiler-Keith-Cooper/dp/012088478X)


Python internals and advanced:
* [Inside The Python Virtual Machine](https://leanpub.com/insidethepythonvirtualmachine)
* [Intermediate Python](https://leanpub.com/intermediatepython)